## Video-52 (ch46): Cookies in Django 5
---
---

```python

HttpResponse.set_cookie(
    key,
    value='',
    max_age = None,  # max_age = 60*60*24*2  -> for 2 days
    expires = None   # expires = datetime.utcnow() + timedelta(days = 2)
    path = '/',     # path = "/home"
    domain = None,  # domain = "google.com"
    secure = False,
    httponly = False,
    samesite = None 
)

```
**domain:**
We set domain only when we need cross domain cookie

**secure**
We set `secure = True` when we want our cookie only transmit through https request

**httponly**
If `httponly = True` then cookie cannot be accessed using javascript in client side or frontend. The cookie is only accessed in backend or server.
Then cookie go to client via submitting by header. We dont need to do explicitly to send cookie.

**Set Cookie:**
- If we close the browser cookie will be deleted
- If we change the value without changing the key then the cookie will be updated
- If we change the key then new cookie with the key will be appended
- We can see cookie : Browser -> Right Click -> Inspect -> Application


**url to hit:**
- Hit `http://127.0.0.1:8000/student/set/` this url to set cookie
- Hit `http://127.0.0.1:8000/student/get/` this url to get cookie
- Hit `http://127.0.0.1:8000/student/del/` this url to delete cookie
- Hit `http://127.0.0.1:8000/student/setsigned/` this url to set cookie
- Hit `http://127.0.0.1:8000/student/getsigned/` this url to get cookie

📂ch46/urls.py

---
```python 

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('student/', include('student.urls'))
]

```

📂student/urls.py

---
```python 

from django.urls import path
from student import views

urlpatterns = [
    path('set/', views.setcookie),
    path('get/', views.getcookie),
    path('del/', views.delcookie),
    path('setsigned/', views.setsignedcookie),
    path('getsigned/', views.getsignedcookie),
]

```

📂student/views.py

---
```python 

from django.shortcuts import render
from datetime import datetime, timedelta, timezone


def setcookie(request):
    response = render(request, 'student/setcookie.html')

    # browser close cookie delete
    response.set_cookie('pay_id', 't123456')  

    # after 1 hour cookie will delete but not in browser close
    # response.set_cookie('pay_id', 't123456', max_age=3600)  

    # after 2 days from now cookie will delete 
    # response.set_cookie('pay_id', 't123456', expires=datetime.now(timezone.utc) + timedelta(days=2))

    return response

def getcookie(request):
    # pay_id = request.COOKIES['pay_id']
    # pay_id = request.COOKIES.get('pay_id', 'default_pay_id') # If cookie value is None then default value will be shown
    pay_id = request.COOKIES.get('pay_id')
    return render(request, 'student/getcookie.html', {'pay_id': pay_id})

def delcookie(request):
    response = render(request, 'student/delcookie.html')
    response.delete_cookie('pay_id')
    return response


def setsignedcookie(request):
    response = render(request, 'student/setsignedcookie.html')
    response.set_signed_cookie('token', 't12345', salt='tk')
    return response

def getsignedcookie(request):
    token = request.get_signed_cookie('token', salt='tk')
    # token = request.get_signed_cookie('token', default='default_token', salt='tk')
    return render(request, 'student/getsignedcookie.html',{'token': token})

```

## Video-53 (ch47): Session in Django 5
---
---

- We can store per site visitor data using session framework
- In session framework data stored in server site but in cookie data is saved in client site or in browser
- This dont destroy at browser close
- This gives security
- django also save data in database so we need to run makemigrations and migrate command

**How Session work:**
- First client request with url from browser
- Request goes to server
- code are written in the backend for setsession 
- Then session is set and data is stored in database session table named `django_session` with `session_key`, `session_data` and `expire_date`
- At the same time cookie is stored in client's browser with `sesionid` key and a `value`
- Browser's sessionid cookie value and database's session_key are same
- This cookie include a key named session id
- When the Client make request next time this session id go with the request
- Then the server recognize that client from the session id


📂ch47/urls.py

---
```python 

from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('student/', include('student.urls'))
]

```


📂student/urls.py

---
```python

from django.urls import path
from .views import setsession, getsession, delsession, flushsession, sessionmethodsinview, sessionmethodsintemplate, sessionclear, settestcookie, checktestcookie, deltestcookie

urlpatterns = [
    path('set/', setsession, name="setsession"),
    path('get/', getsession, name="getsession"),
    path('del/', delsession, name="delsession"),
    path('flush/', flushsession, name="flushsession"),
    path('inview/', sessionmethodsinview, name="sessionmethodsinview"),
    path('intemplate/', sessionmethodsintemplate,
         name="sessionmethodsintemplate"),
    path('clear/', sessionclear, name="sessionclear"),
    
    path('settest/', settestcookie, name="settestcookie"),
    path('checktest/', checktestcookie, name="checktestcookie"),
    path('deltest/', deltestcookie, name="deltestcookie"),
]

```

- Hit `http://127.0.0.1:8000/student/set/` url to set session


📂student/views.py

---
```python

from django.shortcuts import render
from datetime import datetime, timedelta, timezone


def setsession(request):
    request.session['fname'] = 'Pritom'
    request.session['lname'] = 'Karmoker'

    # value in seconds => 10 sec
    # we can not delete this with delete and with flush. We need to use clear_expired
    # request.session.set_expiry(10)

    # session expire in browser close
    # request.session.set_expiry(0)
    return render(request, 'student/setsession.html')


def getsession(request):
    # first_name = request.session['fname']
    # first_name = request.session.get('fname')
    # first_name = request.session.get('fname','Default value')

    first_name = request.session.get('fname')
    last_name = request.session.get('lname')
    return render(request, 'student/getsession.html', {'fname': first_name, 'lname': last_name})


def delsession(request):
    if 'fname' in request.session:
        del request.session['fname']  # This do not fully remove session
    if 'lname' in request.session:
        del request.session['lname']
    return render(request, 'student/delsession.html')


def flushsession(request):
    request.session.flush()     # This fully remove session so used for user logout
    return render(request, 'student/flushsession.html')


def sessionmethodsinview(request):
    keys = request.session.keys()
    print(keys) # output : dict_keys(['fname', 'lname'])

    items = request.session.items()
    print(items) # output : dict_keys([('fname', 'Pritom'), ('lname', 'Karmoker')])

    # if age is set previously than it comes to age variable
    # if age is not set previously than it will set 
    age = request.session.setdefault('age', 31)
    print(age) # output : 31

    session_cookies_age = request.session.get_session_cookie_age()
    print("session_cookies_age : ", session_cookies_age) # Output : session_cookies_age : 1209600

    expire_age = request.session.get_expiry_age()
    print("expire_age : ", expire_age)  # Output : expire_age : 1209600

    expire_date = request.session.get_expiry_date()
    print("expire_date : ", expire_date)  # Output : expire_date : 2026-02-12 09:46:17071363+00:00

    expire_at_browser_close = request.session.get_expire_at_browser_close()
    print("expire_at_browser_close : ", expire_at_browser_close)    # Output: expire_at_browser_close : False

    return render(request, 'student/sessionmethodsinview.html')


def sessionmethodsintemplate(request):
    keys = request.session.keys()
    items = request.session.items()
    age = request.session.setdefault('age', 31)
    return render(request, 'student/sessionmethodsintemplate.html', {'keys': keys, 'items': items})


def sessionclear(request):
    request.session.clear_expired()
    return render(request, 'student/sessionclear.html')


def settestcookie(request):
    request.session.set_test_cookie()
    return render(request, 'student/settestcookie.html')


def checktestcookie(request):
    print(request.session.test_cookie_worked())
    check = request.session.test_cookie_worked()
    return render(request, 'student/checktestcookie.html',{'check': check})


def deltestcookie(request):
    request.session.delete_test_cookie()
    return render(request, 'student/deltestcookie.html')

```